In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
# import mpl_toolkits.mplot3d as Axes3D

%matplotlib inline

In [2]:
dir(tf)

['AggregationMethod',
 'Assert',
 'AttrValue',
 'ConfigProto',
 'DESCRIPTOR',
 'DType',
 'Dimension',
 'Event',
 'FIFOQueue',
 'FixedLenFeature',
 'FixedLenSequenceFeature',
 'FixedLengthRecordReader',
 'GPUOptions',
 'GRAPH_DEF_VERSION',
 'GRAPH_DEF_VERSION_MIN_CONSUMER',
 'GRAPH_DEF_VERSION_MIN_PRODUCER',
 'Graph',
 'GraphDef',
 'GraphKeys',
 'GraphOptions',
 'HistogramProto',
 'IdentityReader',
 'IndexedSlices',
 'InteractiveSession',
 'LogMessage',
 'NameAttrList',
 'NoGradient',
 'NodeDef',
 'OpError',
 'Operation',
 'OptimizerOptions',
 'PaddingFIFOQueue',
 'Print',
 'QUANTIZED_DTYPES',
 'QueueBase',
 'RandomShuffleQueue',
 'ReaderBase',
 'RegisterGradient',
 'RegisterShape',
 'Session',
 'SessionLog',
 'SparseTensor',
 'SparseTensorValue',
 'Summary',
 'TFRecordReader',
 'Tensor',
 'TensorShape',
 'TextLineReader',
 'VarLenFeature',
 'Variable',
 'WholeFileReader',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 